In [2]:
from random import randint as rnd, choice
import tkinter as tk
import math
import time


root = tk.Tk()
root.geometry('800x600')                    #размер окна 
root.title("Gun Game With Points")          #заголовок окна
canv = tk.Canvas(root, width = 800, height = 600, bg = 'white')
canv.pack()

class bullet():
    def __init__(self):
        self.x = 20                         #координаты центра пушки
        self.y = 450
        self.r = 10                         #радиус по умолчанию 10   
        self.vx = 0
        self.vy = 0
        self.lifetime = 150                 #время жизни
        self.color = choice(['blue', 'green', 'red', 'brown'])
        self.id = canv.create_oval([self.x - self.r, self.y - self.r],
                                   [self.x + self.r, self.y + self.r],
                                   fill = self.color)
        
    def move(self):
        dt = 0.2                            #квант времени
        k = 0.7                             #коэффициент гашения
        if (self.y >= 580):                 #ускорение свободного падения
            g = 0
        else:
            g = 8

        if (self.x > 790) or (self.x < 10):
            self.vx *= -1
        elif (self.y > 580):
            self.y = 580
            self.vy *= -k
            self.vx *= k

        self.x += self.vx*dt
        self.y += self.vy*dt
        self.vy += g*dt

        canv.coords(self.id,
                    self.x - self.r, self.y - self.r,
                    self.x + self.r, self.y + self.r)

        self.lifetime -= 1

    def hit_test(self, obj):
        if ((self.x - obj.x)**2 + (self.y - obj.y)**2) <= (obj.r + self.r)**2:
            return True
        return False


class target():
    def __init__(self):
        self.set = 0
        self.x = 0
        self.y = 0
        self.r = 0
        self.vx = 0
        self.vy = 0
        self.id = canv.create_oval(0, 0, 0, 0)

    def new_target(self):
        self.set = 1
        self.x = rnd(600, 780)
        self.y = rnd(300, 550)
        self.r = rnd(10, 50)
        self.vx = rnd(20, 70)
        self.vy = rnd(20, 70)
        canv.coords(self.id,
                    self.x - self.r, self.y - self.r,
                    self.x + self.r, self.y + self.r)
        canv.itemconfig(self.id, fill = 'red')

    def hit(self):
        canv.coords(self.id, 0, 0,
                             0, 0)
        
    def move(self):
        if self.set:
            dt = 0.2                            #квант времени

            if self.x > 790:
                self.vx *= -1
                self.x = 790
            elif self.x < 600:
                self.vx *= -1
                self.x = 600
            elif self.y > 580:
                self.vy *= -1
                self.y = 580
            elif self.y < 20:
                self.vy *= -1
                self.y = 20

            self.x += self.vx*dt
            self.y += self.vy*dt

            canv.coords(self.id,
                        self.x - self.r, self.y - self.r,
                        self.x + self.r, self.y + self.r)


class gun():
    def __init__(self):
        self.fire_power = 20
        self.fire_on = 0
        self.angle = 0
        self.id = canv.create_line([20, 450], [40, 450], width = 3, arrow = tk.LAST)
        
    def fire_start(self, event):
        self.fire_on = 1

    def fire_end(self, event):
        global bullets, shots
        shots += 1
        new_bullet = bullet()
        bullets += [new_bullet]
        new_bullet.vx = self.fire_power * math.cos(self.angle)
        new_bullet.vy = self.fire_power * math.sin(self.angle)
        self.fire_on = 0
        self.fire_power = 20

    def targetting(self, event = 0):
        if event:
            self.angle = math.atan((event.y - 450) / (event.x - 20))
        canv.coords(self.id, 20, 450,
                    20 + self.fire_power * math.cos(self.angle),
                    450 + self.fire_power * math.sin(self.angle))

    def power_up(self):
        if self.fire_on: 
            if (self.fire_power < 100):
                self.fire_power += 1
            canv.itemconfig(self.id, fill = 'orange')
        else:
            canv.itemconfig(self.id, fill = 'black')
        
            
            
perfomance = canv.create_text(400, 300, text = '', font = '28')
score = canv.create_text(20, 20, text = '', font = '28')
t1 = target()
t2 = target()
g1 = gun()
bullets = []
shots = 0
points = 0

def new_game(event = ''):
    global g1, t1, t2, perfomance, bullets, shots, points
    shots = 0
    canv.itemconfig(perfomance, text = '')

    canv.bind('<Button-1>', g1.fire_start)
    canv.bind('<ButtonRelease-1>', g1.fire_end)
    canv.bind('<Motion>', g1.targetting)

    t1.new_target()
    t2.new_target()

    while t1.set or t2.set or bullets:
        t1.move()
        t2.move()
        for b in bullets:
            b.move()
            if b.hit_test(t1) and t1.set:
                t1.set = 0
                t1.hit()
                points += 1
            if b.hit_test(t2) and t2.set:
                t2.set = 0
                t2.hit()
                points += 1
            if (t1.set == 0) and (t2.set == 0):
                canv.bind('<Button-1>', '')
                canv.bind('<ButtonRelease-1>', '')
                canv.itemconfig(perfomance, text = 'Вы уничтожили обе цели за ' + str(shots) + ' выстрелов') 
            if b.lifetime == 0:
                bullets.remove(b)
                canv.delete(b.id)
        canv.itemconfig(score, text = str(points)) 
        time.sleep(0.03)
        g1.targetting()
        g1.power_up()
        canv.update()
    root.after(750, new_game)

new_game()

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\1\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\1\anaconda3\lib\tkinter\__init__.py", line 804, in callit
    func(*args)
  File "<ipython-input-2-52a321c5f205>", line 167, in new_game
    t1.move()
  File "<ipython-input-2-52a321c5f205>", line 103, in move
    canv.coords(self.id,
  File "C:\Users\1\anaconda3\lib\tkinter\__init__.py", line 2761, in coords
    self.tk.call((self._w, 'coords') + args))]
_tkinter.TclError: invalid command name ".!canvas"
